<a href="https://colab.research.google.com/github/SBoo9/godot/blob/master/Bidirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Fake News Classifier

In [2]:
import pandas as pd


In [3]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [4]:
df=pd.read_csv('train.csv',engine='python',on_bad_lines='skip')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

,0
id,0
title,51
author,209
text,4
label,0


In [7]:
df=df.dropna()

In [8]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [9]:
df.shape

(1715, 5)

### GET INDEPENDENT FEATURE

In [10]:
X=df.drop('label',axis=1)

In [11]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [12]:
y=df['label']

### CHECK IF DATASET IS BALANCED OR NOT

In [13]:
y.value_counts()

,count
label,
0,966
1,749


### MODEL

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.17.1'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense,Bidirectional

In [17]:
voc_size=5000

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
messages=X.copy()

In [21]:
messages.reset_index(inplace=True)

In [22]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  print(i)
  review=re.sub('^[a-zA-z]'," ",messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
corpus

['ous dem aide: didn’t even see comey’ letter jason chaffetz tweet',
 'lynn: hillari clinton, big woman campu - breitbart',
 'hy truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'ranian woman jail fiction unpublish stori woman stone death adulteri',
 'acki mason: hollywood would love trump bomb north korea lack tran bathroom (exclus video) - breitbart',
 'enoît hamon win french socialist party’ presidenti nomin - new york time',
 'back-channel plan ukrain russia, courtesi trump associ - new york time',
 'bama’ organ action partner soros-link ‘indivisible’ disrupt trump’ agenda',
 'bc comedi sketch "real housew isis" caus outrag',
 'ussian research discov secret nazi militari base ‘treasur hunter’ arctic [photos]',
 'offici see link trump russia',
 'e: yes, paid govern troll social media, blogs, forum websit',
 'n major leagu soccer, argentin find home success - new york time',
 'ell fargo chief abruptli step - new york time',
 'nonym donor pay $2.5 million releas

In [24]:
one_hot_repr=[one_hot(words,voc_size) for words in corpus]

In [25]:
print(one_hot_repr)

[[635, 4426, 2850, 3818, 815, 343, 1957, 4333, 4534, 3037, 2393], [3966, 4174, 2968, 4615, 2099, 4508, 2699], [2988, 134, 4406, 4551, 3057], [2045, 1987, 2377, 228, 2547, 1368, 1887], [267, 2099, 3293, 2090, 2953, 1630, 2099, 3314, 1424, 3243], [2934, 2367, 4091, 650, 2466, 2229, 3371, 254, 141, 1619, 3657, 4144, 1876, 4679, 2699], [3412, 570, 1063, 3098, 1686, 4226, 3869, 4827, 978, 4598, 111], [650, 41, 2745, 2435, 991, 3387, 2229, 128, 978, 4598, 111], [1265, 2866, 1109, 2443, 3513, 3343, 2937, 3349, 3400, 2972], [3531, 4758, 3850, 4177, 1536, 1692, 820, 3669], [4866, 2758, 1454, 2203, 1826, 346, 3636, 2660, 4535, 1646, 1350], [897, 343, 3343, 2229, 991], [2156, 4696, 447, 1197, 1513, 4145, 3253, 4974, 2878, 1082], [3009, 389, 3143, 3246, 2048, 4463, 115, 1117, 978, 4598, 111], [3080, 3961, 4402, 4814, 1636, 978, 4598, 111], [4423, 959, 2177, 2094, 1099, 2379, 2436, 1832, 2355, 3584, 3368, 4584], [4774, 4287, 3448], [1940, 1464, 3045, 963, 2229, 3324, 3197, 2699], [235, 468, 2968, 3

In [26]:
sent_length=20
embedded_docs=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)

In [27]:
embedded_docs

array([[   0,    0,    0, ..., 4534, 3037, 2393],
       [   0,    0,    0, ..., 2099, 4508, 2699],
       [   0,    0,    0, ..., 4406, 4551, 3057],
       ...,
       [   0,    0,    0, ...,  978, 4598,  111],
       [   0,    0,    0, ...,  978, 4598,  111],
       [   0,    0,    0, ..., 1834,   30, 1870]], dtype=int32)

In [28]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [30]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=2)

In [34]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - accuracy: 0.5663 - loss: 0.6765 - val_accuracy: 0.8092 - val_loss: 0.5829
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - accuracy: 0.8374 - loss: 0.4695 - val_accuracy: 0.8675 - val_loss: 0.3436
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.9602 - loss: 0.1405 - val_accuracy: 0.8728 - val_loss: 0.3920
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.9810 - loss: 0.0775 - val_accuracy: 0.8339 - val_loss: 0.3883
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9900 - loss: 0.0424 - val_accuracy: 0.8728 - val_loss: 0.3798
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9959 - loss: 0.0222 - val_accuracy: 0.8975 - val_loss: 0.3577
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9934 - loss: 0.0314 - val_accuracy: 0.8869 - val_loss: 0.4181
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9986 - loss: 0.0095 - val_accuracy: 0.8905 

In [45]:
y_pred=model.predict(X_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [48]:
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix,accuracy_score

In [47]:
confusion_matrix(y_test,y_pred.round())

array([[288,  22],
       [ 48, 208]])

In [49]:
accuracy_score(y_test,y_pred.round())

0.8763250883392226